Install MySql Connector. 


In [0]:
#pip install mysql-connector-python

# Import required packages

In [0]:
import mysql.connector
import pandas as pd

# Retrieve data

1.   Create MySql Connection
2.   Retrieve data from all the required tables & store in a dataframe.
3.   Close Connection






In [0]:
def connect_db():
    return mysql.connector.connect(
        host='cpanel.insaid.co',
        database='Capstone1',
        user='student',
        password='student',
        connect_timeout=1000000,
        buffered=True
    )

In [0]:
def execute_query(query, connection):
    df = pd.read_sql(query, con = connection)
    return df

In [0]:
def get_data(query):
    df = pd.DataFrame()
    try:
        db_connection = connect_db()
    
        if db_connection.is_connected():
            df = execute_query(query, db_connection)
    
    except Error as e:
        print("Error while connecting to MySQL", e)
        print("Error code:", e.errno)
        print("Error message:", e.msg)
    finally:
        if (db_connection.is_connected()):            
            db_connection.close()
            print("MySQL connection is closed")
    
    return df

In [9]:
# Fetch events data result
sql = "SELECT * FROM events_data where state in ('WestBengal', 'Karnataka', 'Gujarat', 'Bihar', 'Punjab', 'Kerala')"
events_data = get_data(sql)

# Fetch gender result
gender_age_train_full = events_data = get_data("SELECT * FROM gender_age_train")
gender_age_train = events_data = get_data("SELECT * FROM gender_age_train where device_id in (SELECT distinct device_id FROM events_data where state in ('WestBengal', 'Karnataka', 'Gujarat', 'Bihar', 'Punjab', 'Kerala'))")

# Fetch phone brand result
phone_brand_device_model_full = get_data("SELECT * FROM phone_brand_device_model")
phone_brand_device_model = get_data("SELECT * FROM phone_brand_device_model where device_id in (SELECT distinct device_id FROM events_data where state in ('WestBengal', 'Karnataka', 'Gujarat', 'Bihar', 'Punjab', 'Kerala'))")


MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed


gender_age_train - Devices and their respective user gender, age and age_group

In [11]:
gender_age_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19032 entries, 0 to 19031
Data columns (total 4 columns):
device_id    19032 non-null int64
gender       19032 non-null object
age          19032 non-null int64
group        19032 non-null object
dtypes: int64(2), object(2)
memory usage: 594.9+ KB


In [12]:
gender_age_train_full.isnull().any()

device_id    False
gender       False
age          False
group        False
dtype: bool

In [13]:
events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19032 entries, 0 to 19031
Data columns (total 4 columns):
device_id    19032 non-null int64
gender       19032 non-null object
age          19032 non-null int64
group        19032 non-null object
dtypes: int64(2), object(2)
memory usage: 594.9+ KB


In [14]:
events_data.isnull().any()

device_id    False
gender       False
age          False
group        False
dtype: bool

In [15]:
events_data.isnull().sum()

device_id    0
gender       0
age          0
group        0
dtype: int64

In [16]:
#missing data
total = events_data.isnull().sum().sort_values(ascending=False)
percent = (events_data.isnull().sum()/events_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)
print(missing_data)

           Total  Percent
group          0      0.0
age            0      0.0
gender         0      0.0
device_id      0      0.0


In [0]:
# null values are not even 1% of total data
events_data.dropna(inplace=True)

In [18]:
events_data.describe(include='all')

,device_id,gender,age,group
count,1.903200e+04,19032,19032.000000,19032
unique,NaN,2,NaN,12
top,NaN,M,NaN,M32-38
freq,NaN,12152,NaN,2433
mean,4.881698e+16,NaN,31.498371,NaN
std,5.336355e+18,NaN,9.867969,NaN
min,-9.221066e+18,NaN,11.000000,NaN
25%,-4.549678e+18,NaN,25.000000,NaN
50%,6.706378e+16,NaN,29.000000,NaN
75%,4.704435e+18,NaN,36.000000,NaN


In [19]:
events_data.head()

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,176515041953473526,M,33,M32-38
2,9032155484127182494,M,31,M29-31
3,7477216237379271436,F,37,F33-42
4,-6876541075223249434,M,75,M39+


In [20]:
phone_brand_device_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19032 entries, 0 to 19031
Data columns (total 3 columns):
device_id       19032 non-null int64
phone_brand     19032 non-null object
device_model    19032 non-null object
dtypes: int64(1), object(2)
memory usage: 446.2+ KB


In [21]:
phone_brand_device_model_full.isnull().any()

device_id       False
phone_brand     False
device_model    False
dtype: bool

phone_brand_device_model - device ids, brand, and models phone_brand: note that few brands are in Chinese

Brand Name	Brand English Mapping
'华为'	'Huawei'
'小米'	'Xiaomi'
'三星'	'Samsung'
'vivo'	'vivo'
'OPPO'	'OPPO'
'魅族'	'Meizu'
'酷派'	'Coolpad'
'乐视'	'LeEco'
'联想 '	'Lenovo'
'HTC'	'HTC'

In [22]:
phone_brand_device_model['phone_brand'].unique()

array(['华为', '小米', '三星', 'LG', 'vivo', '欧比', 'OPPO', 'TCL', '酷派', 'HTC',
       '联想 ', '魅族', '海信', '优米', '欧博信', '天语', '努比亚', '夏新', '奇酷', '锤子',
       '酷比魔方', '语信', '朵唯', '德赛', '乐视', '一加', '华硕', '普耐尔', '易派', '摩托罗拉',
       '优购', '酷珀', '梦米', '爱派尔', '昂达', 'LOGO', '邦华', '聆韵', '中国移动', 'ZUK',
       '诺亚信', '酷比', '艾优尼', '斐讯', 'Lovme', '富可视', '欧新', '美图', '宝捷讯', '青橙',
       '果米', '波导', '诺基亚', '神舟', '糯米', '亿通', '纽曼', '唯米', '百立丰', '沃普丰',
       '虾米', '凯利通', '贝尔丰', '至尊宝', '蓝魔', '青葱', '白米', '大可乐', '尼比鲁', '康佳',
       '乡米', '黑米', '海尔', '优语', '维图', '米歌', '广信', 'E派', '小杨树', '糖葫芦', '西米',
       '谷歌', '长虹', '飞利浦', '惠普', '台电', '大Q', '鲜米', '先锋', 'PPTV', '欧奇'],
      dtype=object)

In [23]:
phone_brand_device_model.head()

,device_id,phone_brand,device_model
0,176515041953473526,华为,G610S
1,-6876541075223249434,小米,MI 2
2,8026504930081700361,小米,MI 3
3,-7216269352973401877,小米,MI 2S
4,3444770318780911055,三星,Galaxy Note 2


In [0]:
brands_in_chinese = ['华为','小米','三星','魅族','酷派','乐视','联想 ','vivo','OPPO','HTC','LG','TCL','LOGO','Lovme','ZUK','PPTV','努比亚','海信','天语','夏新','欧比', '优米', '欧博信', '奇酷', '锤子', '酷比魔方', '语信', '朵唯', '德赛', '一加',
       '华硕', '普耐尔', '易派', '摩托罗拉', '优购', '酷珀', '梦米', '爱派尔', '昂达', '邦华',
       '聆韵', '中国移动', '诺亚信', '酷比', '艾优尼', '斐讯', '富可视', '欧新', '美图', '宝捷讯',
       '青橙', '果米', '波导', '诺基亚', '神舟', '糯米', '亿通', '纽曼', '唯米', '百立丰',
       '沃普丰', '虾米', '凯利通', '贝尔丰', '至尊宝', '蓝魔', '青葱', '白米', '大可乐', '尼比鲁',
       '康佳', '乡米', '黑米', '海尔', '优语', '维图', '米歌', '广信', 'E派', '小杨树', '糖葫芦',
       '西米', '谷歌', '长虹', '飞利浦', '惠普', '台电', '大Q', '鲜米', '先锋', '欧奇']
brands_in_english = ['Huawei','Xiaomi','Samsung','Meizu','Coolpad','LeEco','Lenovo','vivo','OPPO','HTC','LG','TCL','LOGO','Lovme','ZUK','PPTV','Nubia','Hisense','Amagatarai','Amoi','Obi', 'Youmi', 'Obson', 'Qiku', 'Hammer', 'Cubei Cube', 'Letters', 'Duowei', 'Desai', 'One Plus','ASUS', 'Punale', 'Yipai', 'Motorola', 'Yougo', 'Cooper', 'Mengmi', 'Aipair', 'Onda', 'Banghua', 'Lingyun', 'China Mobile', 'Noah',' Coolby ',' Aiyouni ',' Feixun ',' Focus', 'Ou Xin', 'Meitu', 'Projet News','Green Orange', 'Fruit Rice', 'Flyer', 'Nokia', 'Shenzhou', 'Glutinous Rice', 'Yi Tong', 'Newman', 'Wei Mi', 'Bai Li Feng','Wopfung', 'Shrimp', 'Kellystone', 'Bellfonte', 'Extreme Treasure', 'Blue Devils', 'Scallion', 'White Rice', 'Big Cola', 'Nibiru','Kangjia', 'Xianmi', 'Black Rice', 'Haier', 'Youyu', 'Weitu', 'Mi Ge', 'Guangxin', 'E-Pay', 'Little Poplar', ' Candied ','Simi', 'Google', 'Changhong', 'Philips', 'HP', 'Taipower', 'Big Q', 'Fresh Rice', 'Pioneer', 'Ouqi']

df2 = pd.DataFrame({'phone_brand':brands_in_chinese, 'phone_brand_english':brands_in_english})

In [25]:
df2.head()

,phone_brand,phone_brand_english
0,华为,Huawei
1,小米,Xiaomi
2,三星,Samsung
3,魅族,Meizu
4,酷派,Coolpad


In [0]:
phone_brand = phone_brand_device_model.merge(df2, how='left')

In [27]:
phone_brand.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19032 entries, 0 to 19031
Data columns (total 4 columns):
device_id              19032 non-null int64
phone_brand            19032 non-null object
device_model           19032 non-null object
phone_brand_english    19032 non-null object
dtypes: int64(1), object(3)
memory usage: 743.4+ KB


In [33]:
tele_data = events_data.merge(gender_age_train , on='device_id', how='left').merge(phone_brand, on='device_id', how='left')
tele_data.head()

,device_id,gender_x,age_x,group_x,gender_y,age_y,group_y,phone_brand,device_model,phone_brand_english
0,-8076087639492063270,M,35,M32-38,M,35,M32-38,小米,MI 2,Xiaomi
1,176515041953473526,M,33,M32-38,M,33,M32-38,华为,G610S,Huawei
2,9032155484127182494,M,31,M29-31,M,31,M29-31,华为,Mate 7,Huawei
3,7477216237379271436,F,37,F33-42,F,37,F33-42,华为,荣耀6 plus,Huawei
4,-6876541075223249434,M,75,M39+,M,75,M39+,小米,MI 2,Xiaomi


In [29]:
phone_brand['phone_brand_english'].unique()

array(['Huawei', 'Xiaomi', 'Samsung', 'LG', 'vivo', 'Obi', 'OPPO', 'TCL',
       'Coolpad', 'HTC', 'Lenovo', 'Meizu', 'Hisense', 'Youmi', 'Obson',
       'Amagatarai', 'Nubia', 'Amoi', 'Qiku', 'Hammer', 'Cubei Cube',
       'Letters', 'Duowei', 'Desai', 'LeEco', 'One Plus', 'ASUS',
       'Punale', 'Yipai', 'Motorola', 'Yougo', 'Cooper', 'Mengmi',
       'Aipair', 'Onda', 'LOGO', 'Banghua', 'Lingyun', 'China Mobile',
       'ZUK', 'Noah', ' Coolby ', ' Aiyouni ', ' Feixun ', 'Lovme',
       ' Focus', 'Ou Xin', 'Meitu', 'Projet News', 'Green Orange',
       'Fruit Rice', 'Flyer', 'Nokia', 'Shenzhou', 'Glutinous Rice',
       'Yi Tong', 'Newman', 'Wei Mi', 'Bai Li Feng', 'Wopfung', 'Shrimp',
       'Kellystone', 'Bellfonte', 'Extreme Treasure', 'Blue Devils',
       'Scallion', 'White Rice', 'Big Cola', 'Nibiru', 'Kangjia',
       'Xianmi', 'Black Rice', 'Haier', 'Youyu', 'Weitu', 'Mi Ge',
       'Guangxin', 'E-Pay', 'Little Poplar', ' Candied ', 'Simi',
       'Google', 'Changhong', 'Ph

In [31]:
phone_brand['device_model'].nunique()

1092

In [34]:
tele_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19032 entries, 0 to 19031
Data columns (total 10 columns):
device_id              19032 non-null int64
gender_x               19032 non-null object
age_x                  19032 non-null int64
group_x                19032 non-null object
gender_y               19032 non-null object
age_y                  19032 non-null int64
group_y                19032 non-null object
phone_brand            19032 non-null object
device_model           19032 non-null object
phone_brand_english    19032 non-null object
dtypes: int64(3), object(7)
memory usage: 1.6+ MB
